In [1]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
with open("fixed_log.log") as f:
    log = f.readlines()

In [3]:
#for i, entry in enumerate(log):
 #   requests.post(f"https://localhost:9200/gluttonhoneypot/_doc/{i+1}", json=json.loads(entry), auth=('admin', 'admin'), verify=False)

In [7]:
log[3:5]

['{"level":"info","ts":1673794463.1364226,"msg":"IP from a known scanner","sensorID":"cd00083f-1221-429d-9d07-49cc5e1c2005","host":"167.94.138.61","scanner":"censys","dest_port":"53482"}\n',
 '{"level":"info","ts":1673794464.202223,"msg":"IP from a known scanner","sensorID":"cd00083f-1221-429d-9d07-49cc5e1c2005","host":"167.94.138.61","scanner":"censys","dest_port":"53482"}\n']

In [12]:
list_of_dicts = []
for line in log[3:]:
    list_of_dicts.append(json.loads(line))

In [13]:
df = pd.DataFrame(list_of_dicts)

In [14]:
df.head()

,level,ts,msg,sensorID,host,scanner,dest_port,handler,src_ip,src_port,...,password,message,direction,error,payload_hash,peer_id,inf_hash,sha256,source,sha
0,info,1.673794e+09,IP from a known scanner,cd00083f-1221-429d-9d07-49cc5e1c2005,167.94.138.61,censys,53482,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,info,1.673794e+09,IP from a known scanner,cd00083f-1221-429d-9d07-49cc5e1c2005,167.94.138.61,censys,53482,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,info,1.673794e+09,IP from a known scanner,cd00083f-1221-429d-9d07-49cc5e1c2005,167.94.138.61,censys,53482,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,info,1.673794e+09,"running Go routines: 35, open files: 8",cd00083f-1221-429d-9d07-49cc5e1c2005,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,info,1.673794e+09,IP from a known scanner,cd00083f-1221-429d-9d07-49cc5e1c2005,167.94.138.61,censys,53482,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
pd.set_option("display.max_rows", None)
print(df.dest_port.value_counts())

23       2416231
2323      177774
3399       41689
37215      34065
23231       4162
80          3808
21          3255
8080        2802
8888        1478
6379        1462
8088        1338
6666        1238
81          1168
808         1163
8443        1150
27017       1119
8998        1109
9050        1064
8081         964
2375         911
3128         755
9999         738
8123         684
5555         679
2376         657
9000         653
8118         591
56575        519
9200         447
8000         401
7443         388
1433         352
465          338
4443         301
82           295
102          289
3306         272
6443         265
9001         264
8001         264
88           261
37777        258
5432         255
1723         252
8089         245
5984         242
10443        234
83           232
5678         230
631          230
8883         228
2096         228
2083         221
3333         221
2000         218
9090         217
139          216
2086         213
2087         2

In [32]:
(df.src_ip.value_counts().to_frame().head(100).sum() / len(list_of_dicts)) * 100

src_ip    72.164882
dtype: float64